In [16]:
from feature_template import driver_stats_fresh_fv, driver_stats_fv, driver
from feast import FeatureStore, RepoConfig
import pandas as pd
from datetime import datetime
import pytz

In [17]:
fs = FeatureStore(repo_path=r"E:/feast_db/my_project/feature_repo")

In [19]:
fs.apply([driver,driver_stats_fv, driver_stats_fresh_fv])

In [23]:
df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

In [25]:
training_df = fs.get_historical_features(
    features=[
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_df=df
).to_df()
print()
print(training_df)


   driver_id           event_timestamp  conv_rate  acc_rate  avg_daily_trips
0       1001 2021-04-12 10:59:42+00:00   0.566946  0.978494              981
1       1002 2021-04-12 08:12:10+00:00   0.786480  0.891738              882
2       1003 2021-04-12 16:40:26+00:00   0.217032  0.866702              170
3       1004 2021-04-12 15:01:12+00:00   0.345235  0.022926              302


In [26]:
fs.list_feature_views()

[<FeatureView(name = driver_hourly_stats_fresh, entities = ['driver'], ttl = 1 day, 0:00:00, stream_source = {
   "type": "PUSH_SOURCE",
   "name": "driver_stats_push_source",
   "batchSource": {
     "type": "BATCH_FILE",
     "timestampField": "event_timestamp",
     "createdTimestampColumn": "created",
     "fileOptions": {
       "uri": "data/driver_stats.parquet"
     },
     "name": "driver_hourly_stats_source"
   }
 }, batch_source = {
   "type": "BATCH_FILE",
   "timestampField": "event_timestamp",
   "createdTimestampColumn": "created",
   "fileOptions": {
     "uri": "data/driver_stats.parquet"
   },
   "name": "driver_hourly_stats_source"
 }, entity_columns = [Field(
     name='driver_id',
     dtype=<PrimitiveFeastType.INT64: 4>,
     description='',
     tags={}
     vector_index=False
     vector_search_metric=''
 )], features = [Field(
     name='conv_rate',
     dtype=<PrimitiveFeastType.FLOAT32: 6>,
     description='',
     tags={}
     vector_index=False
     vector_

In [36]:
online_feature_df = fs.get_online_features(
    features=["driver_hourly_stats:conv_rate", "driver_hourly_stats:acc_rate", "driver_hourly_stats:avg_daily_trips"],
    entity_rows=[{"driver_id": 1005}]
).to_dict()

In [37]:
online_feature_df

{'driver_id': [1005],
 'conv_rate': [0.6906761527061462],
 'avg_daily_trips': [526],
 'acc_rate': [0.6691997647285461]}